# Tile Server Process for Aurora Outputs

In [1]:
import os
import sys

sys.path.append('../')

print(os.getcwd())
print(os.listdir())

import yaml
from PIL import Image
import numpy as np
import torch
import xarray as xr
import matplotlib.pyplot as plt
import cfgrib
from datetime import datetime, timedelta
import time
from subprocess import Popen
from io import BytesIO
import ast
from cartopy import crs as ccrs, feature as cfeature
from datetime import datetime
from tile_server import save_tile, tile_into_folders
from serve import serve

/home/jgib124/aurora/aurora/afwerx
['serve.py', 'tile_server_process.ipynb', '__pycache__', 'aurora_regridding_process.ipynb', 'visualize.py', 'ssd1', 'flask_server.py', 'configs', 'tile_server.py', 'key.pem', 'tiles', 'cert.pem', 'aurora_inference_simple.py']


/home/jgib124/miniconda3/envs/aurora_test/lib/python3.11/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


Wasabi: 
	Endpoint:  https://s3.wasabisys.com/
	Bucket:  aa-archiver
IDrive: 
	Endpoint:  https://l5l2.va.idrivee2-56.com
	Bucket:  archive-mosaics


In [2]:
# Read in regridded data
ds = xr.open_dataset("tiles/combined_era_aurora_regridded.nc", engine='netcdf4')

ds

<xarray.Dataset> Size: 153GB
Dimensions:       (time: 16, y: 4608, x: 7168, level: 13)
Coordinates:
  * level         (level) int64 104B 1000 925 850 700 600 ... 250 200 150 100 50
  * time          (time) datetime64[ns] 128B 2023-01-06 ... 2023-01-09T18:00:00
    rollout_step  (time) int64 128B ...
    longitude     (y, x) float64 264MB ...
    latitude      (y, x) float64 264MB ...
Dimensions without coordinates: y, x
Data variables:
    surf_2t       (time, y, x) float32 2GB ...
    surf_10u      (time, y, x) float32 2GB ...
    surf_10v      (time, y, x) float32 2GB ...
    surf_msl      (time, y, x) float32 2GB ...
    static_z      (time, y, x) float32 2GB ...
    static_slt    (time, y, x) float32 2GB ...
    static_lsm    (time, y, x) float32 2GB ...
    atmos_t       (time, level, y, x) float32 27GB ...
    atmos_u       (time, level, y, x) float32 27GB ...
    atmos_v       (time, level, y, x) float32 27GB ...
    atmos_q       (time, level, y, x) float32 27GB ...
    atmos_z       (time, level, y, x) float32 27GB ...
Attributes:
    regrid_method:  bilinear

In [3]:
ds.time.values

array(['2023-01-06T00:00:00.000000000', '2023-01-06T06:00:00.000000000',
       '2023-01-06T12:00:00.000000000', '2023-01-06T18:00:00.000000000',
       '2023-01-07T00:00:00.000000000', '2023-01-07T06:00:00.000000000',
       '2023-01-07T12:00:00.000000000', '2023-01-07T18:00:00.000000000',
       '2023-01-08T00:00:00.000000000', '2023-01-08T06:00:00.000000000',
       '2023-01-08T12:00:00.000000000', '2023-01-08T18:00:00.000000000',
       '2023-01-09T00:00:00.000000000', '2023-01-09T06:00:00.000000000',
       '2023-01-09T12:00:00.000000000', '2023-01-09T18:00:00.000000000'],
      dtype='datetime64[ns]')

In [4]:
def dt2cal(dt):
    """
    Convert array of datetime64 to a calendar array of year, month, day, hour,
    minute, seconds, microsecond with these quantites indexed on the last axis.

    Parameters
    ----------
    dt : datetime64 array (...)
        numpy.ndarray of datetimes of arbitrary shape

    Returns
    -------
    cal : uint32 array (..., 7)
        calendar array with last axis representing year, month, day, hour,
        minute, second, microsecond
    """

    # allocate output 
    out = np.empty(dt.shape + (7,), dtype="u4")
    # decompose calendar floors
    Y, M, D, h, m, s = [dt.astype(f"M8[{x}]") for x in "YMDhms"]
    out[..., 0] = Y + 1970 # Gregorian Year
    out[..., 1] = (M - Y) + 1 # month
    out[..., 2] = (D - M) + 1 # dat
    out[..., 3] = (dt - D).astype("m8[h]") # hour
    out[..., 4] = (dt - h).astype("m8[m]") # minute
    out[..., 5] = (dt - m).astype("m8[s]") # second
    out[..., 6] = (dt - s).astype("m8[us]") # microsecond
    return out

In [ ]:
# NOTE: either 1D or 3D image
# TODO: check for number of channels and repeat if 1D
serve_dir = "/ssd1/aurora_maptuple_multiframe/serve/v2"
zooms = (5, 10)

for i, t in enumerate(ds.time.values):

    timestamp = dt2cal(t)

    timestamp_str = f"{timestamp[0]}{timestamp[1]:02d}{timestamp[2]:02d}{timestamp[3]:02d}{timestamp[4]:02d}"

    print(t, "---", timestamp_str)

    for var in ['atmos_q', 'atmos_v', 'atmos_u']:
        for l in [0, 3, 5, 12]:
            pressure = ds.level.values[l]
            noun = "{}_{}".format(var, pressure)
            selected_array = ds[var].isel(level=l, time=i).to_numpy()
            tile_into_folders(serve_dir=serve_dir, noun=noun, timestamp=timestamp_str, image=selected_array, zoom=zooms)

    for var in ['surf_msl', 'surf_2t']:
        selected_array = ds[var].isel(time=i).to_numpy()
        tile_into_folders(serve_dir=serve_dir, noun=var, timestamp=timestamp_str, image=selected_array, zoom=zooms)

2023-01-06T00:00:00.000000000 --- 2023010600
Processing image: atmos_q_1000 2023010600
Processing image: atmos_q_700 2023010600
Processing image: atmos_q_500 2023010600
Processing image: atmos_q_50 2023010600
Processing image: atmos_v_1000 2023010600
Processing image: atmos_v_700 2023010600
Processing image: atmos_v_500 2023010600
Processing image: atmos_v_50 2023010600
Processing image: atmos_u_1000 2023010600
Processing image: atmos_u_700 2023010600
Processing image: atmos_u_500 2023010600
Processing image: atmos_u_50 2023010600
Processing image: surf_msl 2023010600
Processing image: surf_2t 2023010600
2023-01-06T06:00:00.000000000 --- 2023010606
Processing image: atmos_q_1000 2023010606
Processing image: atmos_q_700 2023010606
Processing image: atmos_q_500 2023010606
Processing image: atmos_q_50 2023010606
Processing image: atmos_v_1000 2023010606
Processing image: atmos_v_700 2023010606
Processing image: atmos_v_500 2023010606
Processing image: atmos_v_50 2023010606
Processing imag

## Serve generated tiles to an endpoint
- HTTP-based server
- Flask server

In [ ]:
# Generate an SSL Certificate to use temporarily
#  openssl req -x509 -days 1 -newkey rsa:2048 -keyout key.pem -out cert.pem

In [21]:
# HTTP Only Approach
serve_dir = "/ssd1/aurora_maptuple_multiframe/serve"
serve_port = "8888"
serve_address = "100.99.120.135"
cert_path = "cert.pem"
key_path = "key.pem"

proc = Popen('python serve.py --serve_dir "{}" --serve_port {} --serve_address "{}" --certfile "{}" --keyfile "{}"'.format(serve_dir, serve_port, serve_address, cert_path, key_path), shell=True)
print(time.ctime(), 'Serving directory "{}" at port {}'.format(serve_dir, serve_port))
proc.pid

Thu Mar 13 00:53:50 2025 Serving directory "/ssd1/aurora_maptuple_multiframe/serve" at port 8888


629963

Serving on https://100.99.120.135:8888


In [ ]:
lat = 39.833 # Starting lat
lon = -98.585 # Starting lon
zoom = 7 # Starting zoom

# num past and future frames to load, based on start time
# NOTE: future frames load from a different URL, leave future=False
#   and set start time to the most recent time frame desired
frames = "4,1" 
startTime = "20230108T180000Z" # 2023-01-09T18:00:00.000000000

# Time interval between frames in minutes
interval = 360

# Starting split, either mirrors or extends the visual
split = "mirror"
base = "https://map.acmeaom.com/dark/" # Basemap
radar = "https://100.99.120.135:8888/v2/" # Data server

var = "atmos_q_1000"

url = "https://maptuple.mrsv.co/?lat={}&lon={}&zoom={}&frames={}&interval={}&startTime={}&split={}&base={}&radar={}{}/".format(lat, lon, zoom, frames, interval, startTime, split, base, radar, var)
url

'https://maptuple.mrsv.co/?lat=39.833&lon=-98.585&zoom=7&frames=4,1&interval=360&startTime=20230108T180000Z&split=mirror&base=https://map.acmeaom.com/dark/&radar=https://100.99.120.135:8888/v2/atmos_q_1000/'

100.112.208.70 - - [13/Mar/2025 00:53:57] code 404, message File not found
100.112.208.70 - - [13/Mar/2025 00:53:57] "GET /v2/atmos_q_1000/202301081800/7/28/48.png HTTP/1.1" 404 -
100.112.208.70 - - [13/Mar/2025 00:53:57] code 404, message File not found
100.112.208.70 - - [13/Mar/2025 00:53:57] "GET /v2/atmos_q_1000/202301081800/7/28/49.png HTTP/1.1" 404 -
100.112.208.70 - - [13/Mar/2025 00:53:57] code 404, message File not found
100.112.208.70 - - [13/Mar/2025 00:53:57] "GET /v2/atmos_q_1000/202301081800/7/29/48.png HTTP/1.1" 404 -
100.112.208.70 - - [13/Mar/2025 00:53:57] code 404, message File not found
100.112.208.70 - - [13/Mar/2025 00:53:57] "GET /v2/atmos_q_1000/202301081800/7/29/49.png HTTP/1.1" 404 -
100.112.208.70 - - [13/Mar/2025 00:53:58] code 404, message File not found
100.112.208.70 - - [13/Mar/2025 00:53:58] "GET /v2/atmos_q_1000/202301081800/7/25/46.png HTTP/1.1" 404 -
100.112.208.70 - - [13/Mar/2025 00:53:58] code 404, message File not found
100.112.208.70 - - [13/Ma

In [ ]:
# https://maptuple.mrsv.co/?lat=39.833&lon=-98.585&zoom=7&frames=1,1&interval=360&startTime=20230101T000000Z&split=mirror&base=https://map.acmeaom.com/dark/&radar=https://100.99.120.135:8888/v2/atmos_q/
# https://maptuple.mrsv.co/?lat=39.833&lon=-98.585&zoom=7&frames=1,1&interval=360&startTime=20230101T000000Z&split=mirror&base=https://map.acmeaom.com/dark/&radar=https://100.99.120.135:8888/v2/atmos_v/
# https://maptuple.mrsv.co/?lat=39.833&lon=-98.585&zoom=7&frames=1,1&interval=360&startTime=20230101T000000Z&split=mirror&base=https://map.acmeaom.com/dark/&radar=https://100.99.120.135:8888/v2/surf_msl/
# https://maptuple.mrsv.co/?lat=39.833&lon=-98.585&zoom=7&frames=1,1&interval=360&startTime=20230101T000000Z&split=mirror&base=https://map.acmeaom.com/dark/&radar=https://100.99.120.135:8888/v2/atmos_u/
# https://maptuple.mrsv.co/?lat=39.833&lon=-98.585&zoom=7&frames=1,1&interval=360&startTime=20230101T000000Z&split=mirror&base=https://map.acmeaom.com/dark/&radar=https://100.99.120.135:8888/v2/atmos_t/

In [ ]:
Popen('netstat -tulpn | grep {}'.format(serve_port), shell=True)

<Popen: returncode: None args: 'netstat -tulpn | grep 8888'>

(Not all processes could be identified, non-owned process info
 will not be shown, you would have to be root to see it all.)


tcp        0      0 100.99.120.135:8888     0.0.0.0:*               LISTEN      628838/python       


In [19]:
Popen("kill {}".format(proc.pid), shell=True)

<Popen: returncode: None args: 'kill 628837'>

In [20]:
%%sh 
kill 628838

### Flask Server

In [ ]:
import cv2
from flask_cors import CORS
from flask import Flask, send_file
import io

TILE_SIZE = 256
ZOOM = 7

CONUS_Y_SIZE = 4608
CONUS_X_SIZE = 7168

# Aurora size: 720, 1440
# CONUS Size: 4608, 7168

minTileX = 0
minTileY = 0
maxTileX = CONUS_X_SIZE // TILE_SIZE
maxTileY = CONUS_Y_SIZE // TILE_SIZE

minX = minTileX * TILE_SIZE
minY = minTileY * TILE_SIZE
maxX = maxTileX * TILE_SIZE
maxY = maxTileY * TILE_SIZE

var_serve_dr = "atmos_t"

application = Flask(__name__)
CORS(application) # Prevents CORS Errors

In [ ]:
def serve_jpg(image):
    '''Input RGB image and return servable jpg.'''
    output = io.BytesIO()
    image.save(output, format='jpeg')
    output.seek(0, 0)
    return send_file(output, mimetype='image/jpeg')

def serve_png(image):
    '''Input RGB image and return servable png.'''
    output = io.BytesIO()
    image.save(output, format='png')
    output.seek(0, 0)
    return send_file(output, mimetype='image/png')

In [ ]:
def resample(tile, zoom, x, y):
    pass

In [ ]:
@application.route('/v2/ref_d_syn/<YYYYMMDDHHMM>/<zoom>/<x>/<y>.png')
def get_png(YYYYMMDDHHMM, zoom, y, x):
    '''
    Process PNG Tile Requests, checking if tile is available or needs to be resampled

    Parameters
    ----------
    YYYYMMDDHHMM : str
        YYYYMMDDHHMM timestamp of the image.
    zoom : str
        Zoom level of the tile.
    y : str
        Y coordinate of the tile.
    x : str
        X coordinate of the tile.
    '''
    # NOTE: ignoring the date for now, this will come later
    tname = f"{serve_dir}/{zoom}/{x}/{y}.png"
    if os.path.isfile(tname):
        return send_file(tname)
    else:
        # Needs to get resampled to required zoom level
        # Will exist in zoom 7
        tname = f"{serve_dir}/7/{x}/{y}.png"
        if os.path.isfile(tname):
            tile = cv2.imread(tname)
            resampled = resample(tile, zoom, x, y)
            return serve_png(resampled)
        else:
            raise FileNotFoundError(f"Tile does not exist in Zoom 7: {tname}")

### Code for finding updates

In [ ]:
def get_updates(host : str, back_steps : int = 0, interval : int = 360, set_now : datetime = None):
    '''
    Returns the most current URL if data is present in the current or previous hour. 
    Will also return the hex UTC timestamp for provided URL

    Parameters
    ----------
    host : str
        Hostname of the server, e.g. MRMS, GFS, forecasts, etc.
    back_steps : int
        Timesteps to go back into time.
    interval : int = 360
        Timestep interval between updates in minutes
    set_now : datetime.datetime = None
        Set a specific datetime to check from. Defaults to None to use datetime.utcnow(), else
        accepts a datetime object of the format datetime(year, month, day, hour, minute, second)
    '''
    if set_now is None:
        now = datetime.utcnow()
    else:
        now = set_now

    # UTC for the nearest interval
    now = now - timedelta(minutes=(back_steps*interval + now.minute % interval),
                                    seconds=now.second, microseconds=now.microsecond)
    
    since_1970 = now - datetime(1970, 1, 1)
    
    timestamp = hex(int(since_1970.total_seconds()))

    print("Hex Timestamp: ", timestamp)
    print("Now Timestamp: ", now)
    print("Time since 1970: ", since_1970)


get_updates("fake_url", set_now=datetime(2023, 1, 1, 7))